In [2]:
import numpy as np
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt

from time import time

from common import *
from generate import *
from filter_kruskal import *

In [3]:
common.run(args="--help")

Allowed options:
  -h [ --help ]                         produce help message
  -m [ --mst-kernel ] arg (=1)          Kernel used calculating MST. 0 = 
                                        regular kruskal, 1 = filter-kruskal
  -s [ --sort-kernel ] arg (=0)         Kernel used for sort() [int]
  -p [ --partition-kernel ] arg (=1)    Kernel used for partition() [int]
  -f [ --filter-kernel ] arg (=1)       Kernel used for filter() [int]
  -c [ --compress-kernel ] arg (=1)     Kernel used for compress() [int]
  -k [ --kruskal-threshold ] arg (=500000)
                                        Kernel used for compress() [int]
  -i [ --inputfile ] arg                input file containing graph data. 
                                        Filepath must be relative to Ex2/
  -t [ --ouputfile_timings ] arg        Output file for timings. Filepath must 
                                        be relative to Ex2/



In [17]:
args = f"--inputfile input_data/barabasi_dense_4000_3600.csv --mst-kernel 1 -s 1 -k 4"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv", args=args, print_output=True)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv --inputfile input_data/barabasi_dense_4000_3600.csv --mst-kernel 1 -s 1 -k 4
7918200 Edges loaded

Benchmark results:
Initialize	µ=0.105962s	sigma=0s	total=0.105962	num_calls=1
Kruskal()	µ=1.83443s	sigma=0s	total=1.83443	num_calls=1
compress()	µ=0.0292994s	sigma=0s	total=0.0292994	num_calls=1
filter()	µ=0.0216864s	sigma=0s	total=0.0216864	num_calls=1
grow MST	µ=0.0252684s	sigma=0s	total=0.0252684	num_calls=1
partition()	µ=0.0943955s	sigma=0s	total=0.0943955	num_calls=1
sort()	µ=1.80915s	sigma=0s	total=1.80915	num_calls=1
sort-assembly	µ=1.79659s	sigma=0s	total=1.79659	num_calls=1
sort-init	µ=0.0111644s	sigma=0s	total=0.0111644	num_calls=1
sort-sort	µ=6.3342e-05s	sigma=0s	total=6.3342e-05	num_calls=1
total	µ=2.09063s	sigma=0s	total=2.09063	num_calls=1


,tag,average,std_dev,total,num_calls
0,Initialize,0.105962,0,0.105962,1
1,Kruskal(),1.834430,0,1.834430,1
2,compress(),0.029299,0,0.029299,1
3,filter(),0.021686,0,0.021686,1
4,grow MST,0.025268,0,0.025268,1
5,partition(),0.094395,0,0.094395,1
6,sort(),1.809150,0,1.809150,1
7,sort-assembly,1.796590,0,1.796590,1
8,sort-init,0.011164,0,0.011164,1
9,sort-sort,0.000063,0,0.000063,1


In [21]:
args = f"--inputfile input_data/barabasi_dense_4000_3600.csv --mst-kernel 1 -s 2 -k 4"
df = common.run_benchmark(common.OUT_DIR / "timing_results.csv", args=args, print_output=True)
df

Running benchmark with ./home/hiti/workspace/GPU_ArchComp/Ex2/build/src/ex2 --ouputfile_timings /home/hiti/workspace/GPU_ArchComp/Ex2/out/timing_results.csv --inputfile input_data/barabasi_dense_4000_3600.csv --mst-kernel 1 -s 2 -k 4
7918200 Edges loaded

Benchmark results:
Initialize	µ=0.0994715s	sigma=0s	total=0.0994715	num_calls=1
Kruskal()	µ=0.189729s	sigma=0s	total=0.189729	num_calls=1
compress()	µ=0.0292113s	sigma=0s	total=0.0292113	num_calls=1
filter()	µ=0.0204952s	sigma=0s	total=0.0204952	num_calls=1
grow MST	µ=0.0242725s	sigma=0s	total=0.0242725	num_calls=1
partition()	µ=0.0896516s	sigma=0s	total=0.0896516	num_calls=1
sort()	µ=0.165441s	sigma=0s	total=0.165441	num_calls=1
total	µ=0.431054s	sigma=0s	total=0.431054	num_calls=1


,tag,average,std_dev,total,num_calls
0,Initialize,0.099472,0,0.099472,1
1,Kruskal(),0.189729,0,0.189729,1
2,compress(),0.029211,0,0.029211,1
3,filter(),0.020495,0,0.020495,1
4,grow MST,0.024272,0,0.024272,1
5,partition(),0.089652,0,0.089652,1
6,sort(),0.165441,0,0.165441,1
7,total,0.431054,0,0.431054,1
